In [1]:
import pandas as pd


df = pd.read_csv('./movies.csv')

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 5.6 MB/s 
     |████████████████████████████████| 448 kB 34.7 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
from konlpy.tag import Okt

okt = Okt()

df['token_final'] = df.overview.apply(okt.nouns)

In [4]:
print(len(df))
df.head()

1782


,Unnamed: 0,title,overview,vote_average,vote_count,label,token_final
0,0,용감한 자가 신부를 데려가리,영국에서 유학중인 라즈 샤룩 칸 와 인도 처녀 심란 까졸 심란은 부모님이 정해주...,8.7,3215,True,"[영국, 유학, 라즈, 샤룩, 칸, 인도, 처녀, 심란, 까졸, 심란, 부모님, 정..."
1,1,쇼생크 탈출,촉망받는 은행 간부 앤디 듀프레인은 아내와 그녀의 정부를 살해했다는 누명을 쓴다 ...,8.7,20082,True,"[촉망, 은행, 간부, 앤디, 듀, 프레, 아내, 그녀, 정부, 살해, 누명, 주변..."
2,2,대부,시실리에서 이민온 뒤 정치권까지 영향력을 미치는 거물로 자리잡은 돈 꼴레오네는 갖...,8.7,15045,True,"[시실리, 이민, 온, 뒤, 정치권, 영향력, 거물, 자리, 돈, 꼴레오네, 갖가지..."
3,3,쉰들러 리스트,2차 세계대전 당시 독일군이 점령한 폴란드 시류에 맞춰 자신의 성공을 추구하는 기...,8.6,12007,True,"[차, 세계대전, 당시, 독일군, 점령, 폴란드, 시류, 자신, 성공, 추구, 기회..."
4,4,에반게리온 3.0+1.01,에반게리온 신극장판의 제4탄이자 완결편 미사토와 반네르프 조직 빌레는 코어와 닮은...,8.6,366,True,"[에반게리온, 극장판, 제, 완결, 편, 미사, 토, 르프, 조직, 빌레, 코어, ..."


In [5]:
df_drop = df[['token_final', 'label']]
df_drop.head()

,token_final,label
0,"[영국, 유학, 라즈, 샤룩, 칸, 인도, 처녀, 심란, 까졸, 심란, 부모님, 정...",True
1,"[촉망, 은행, 간부, 앤디, 듀, 프레, 아내, 그녀, 정부, 살해, 누명, 주변...",True
2,"[시실리, 이민, 온, 뒤, 정치권, 영향력, 거물, 자리, 돈, 꼴레오네, 갖가지...",True
3,"[차, 세계대전, 당시, 독일군, 점령, 폴란드, 시류, 자신, 성공, 추구, 기회...",True
4,"[에반게리온, 극장판, 제, 완결, 편, 미사, 토, 르프, 조직, 빌레, 코어, ...",True


In [6]:
from gensim.models import Word2Vec

In [7]:
embedding_model = Word2Vec(
    df_drop['token_final'],
    sg = 1,
    size = 100,
    window = 2,
    min_count = 1,
    workers = 4
    )
print(embedding_model)

Word2Vec(vocab=14182, size=100, alpha=0.025)


In [8]:
model_result = embedding_model.wv.most_similar('은행')
print(model_result)

[('무기', 0.9997416138648987), ('환상', 0.9997378587722778), ('린', 0.9997240304946899), ('달러', 0.9997202157974243), ('전체', 0.9997150301933289), ('도중', 0.9997121691703796), ('칼', 0.9997060298919678), ('마련', 0.9997059106826782), ('노', 0.999700665473938), ('컴퓨터', 0.999699592590332)]


In [9]:
from gensim.models import KeyedVectors

In [10]:
embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v')

model_result = loaded_model.most_similar('은행')
print(model_result)

[('무기', 0.9997416138648987), ('환상', 0.9997378587722778), ('린', 0.9997240304946899), ('달러', 0.9997202157974243), ('전체', 0.9997150301933289), ('도중', 0.9997121691703796), ('칼', 0.9997060298919678), ('마련', 0.9997059106826782), ('노', 0.999700665473938), ('컴퓨터', 0.999699592590332)]


In [11]:
from numpy.random import RandomState

In [12]:
rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

In [13]:
import torchtext
from torchtext.legacy.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

In [14]:
from torchtext.legacy.data import TabularDataset

In [15]:
import re

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print('Train:', train[0].text, train[0].label)
print('Validation:', validation[0].text, validation[0].label)

Train: ['친구', '귀지', '못', '학교', '적응', '소년', '짐', '스타크', '술', '경찰서', '짐', '경찰서', '밤길', '온', '디', '강아지', '총', '온', '소년', '플라토', '청소년', '경찰', '레이', '짐', '부모', '대한', '불만', '것', '알', '어려움', '자기', '말', '며칠', '후', '새', '학교', '등교', '짐', '디', '버즈', '일당', '패거리', '버즈', '짐', '만', '음', '경주', '하자', '도전'] True
Validation: ['시실리', '이민', '온', '뒤', '정치권', '영향력', '거물', '자리', '돈', '꼴레오네', '갖가지', '고민', '호소', '사람', '문제', '해결', '대부', '한편', '솔로', '소라', '인물', '꼴레오네', '라이벌', '탓', '리아', '패밀리', '마약', '사업', '제안', '돈', '꼴레오네', '마약', '사업', '참여', '하자', '돈', '꼴레오네', '저격', '그', '중상', '사경', '그', '뒤', '돈', '꼴레오네', '아들', '소니', '조직', '총', '동원', '다른', '패밀리', '피', '전쟁', '시작', '가족', '사업', '대학', '진학', '뒤', '인텔리', '막내', '아들', '마이클', '아버지', '총격', '당한', '뒤', '아버지', '구', '위해', '위험천만', '협상', '자리'] True


In [16]:
t = f = 0
i = 0
while True:
    try:
        if train[i].label == 'True':
            t += 1
        else:
            f += 1
        i += 1
    except:
        break

i = 0
while True:
    try:
        if validation[i].label == 'True':
            t += 1
        else:
            f += 1
        i += 1
    except:
        break

print(t, f)

909 873


In [17]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

In [18]:
vectors = Vectors(name="data/petitions_tokens_w2v")

TEXT.build_vocab(train, vectors=vectors, min_freq=1, max_size=None)

LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

100%|██████████| 14182/14182 [00:00<00:00, 18506.40it/s]


임베딩 벡터의 개수와 차원 : torch.Size([12789, 100])


## TextCNN 모델 설계

In [19]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):

    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):

        super(TextCNN, self).__init__()

        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)


    def forward(self, x):

        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]

        fc_x = torch.cat(pool_x, dim=1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.dropout(fc_x)

        logit = self.fc(fc_x)

        return logit

In [20]:
def train(model, device, train_itr, optimizer):

    model.train()
    corrects, train_loss = 0.0, 0

    for batch in train_itr:

        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        optimizer.zero_grad()
        logit = model(text)

        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        result = torch.max(logit, 1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()


    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

In [21]:
def evaluate(model, device, itr):

    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:

        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit, 1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()


    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects / len(itr.dataset)

    return test_loss, accuracy

In [22]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

TextCNN(
  (embed): Embedding(12789, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 7+1):
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))

    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))

    if val_acc > best_test_acc:
        best_test_acc = val_acc
        print('model saves at {} accuracy'.format(best_test_acc))
        torch.save(model.state_dict(), 'TextCNN_Best_Validation')

    print('-------------------------------------------------------------------------------')

Train Epoch: 1 	 Loss: 0.08735288945020164 	 Accuracy: 50.63113784790039%
Valid Epoch: 1 	 Loss: 0.08770369997854983 	 Accuracy: 47.191009521484375%
model saves at 47.191009521484375 accuracy
-------------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08546049272194818 	 Accuracy: 57.433380126953125%
Valid Epoch: 2 	 Loss: 0.08655850103731906 	 Accuracy: 54.7752799987793%
model saves at 54.7752799987793 accuracy
-------------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.06866212797365549 	 Accuracy: 80.01402282714844%
Valid Epoch: 3 	 Loss: 0.08433327530876975 	 Accuracy: 57.30337142944336%
model saves at 57.30337142944336 accuracy
-------------------------------------------------------------------------------
Train Epoch: 4 	 Loss: 0.026830313647581268 	 Accuracy: 94.60028076171875%
Valid Epoch: 4 	 Loss: 0.0925208196211397 	 Accuracy: 56.741573333740234%
----------------------------------------